In [12]:
import numpy as np
from rouge import Rouge

In [1]:
from fastapi import FastAPI
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import BertForMaskedLM
import tensorflow as tf

from sklearn.model_selection import KFold
from rouge import Rouge


# tokenizer = AutoTokenizer.from_pretrained('facebook/bart-base')
# model = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-base', from_tf=True)

model = AutoModelForSeq2SeqLM.from_pretrained('../models/fine_tuned/fine_tuned_model_v2.keras/', from_tf=True)

#tokenizer = BertForMaskedLM.from_pretrained('../models/fine_tuned/fine_tuned_model_v1.keras/', from_tf=True)
tokenizer = AutoTokenizer.from_pretrained('../models/fine_tuned/fine_tuned_model_v2.keras/', from_tf=True)




2024-04-25 16:25:36.102480: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-04-25 16:25:36.102503: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-04-25 16:25:36.102508: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-04-25 16:25:36.102527: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-25 16:25:36.102539: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
All TF 2.0 model weights were used when initializing BartForConditionalGeneration.

Some weights of BartForConditionalGeneration were not initialized from the TF 2.0 model and are ne

In [36]:

# Generate summaries using the fine-tuned model
new_text = """

The parents of an Israeli-American hostage in Gaza have urged him to "stay strong" and "survive" after Hamas released a proof-of-life video.
Hersh Goldberg-Polin, 23, is seen without his lower left arm in the short clip. It was blown off during Hamas's 7 October attack on southern Israel.
The footage is undated, but he says he has been held for almost 200 days.
In response, his mother and father appealed for more to be done to secure a new hostage release deal.
They urged Israel, Hamas and mediators the US, Egypt and Qatar to "get a deal done" to reunite "all of us with our loved ones and end the suffering in this region".
Speaking under duress in the video posted on Hamas's Telegram account on Wednesday, Mr Goldberg-Polin said he needed medical help and was critical of the Israeli government's attempts to negotiate the hostages' return.
Weeks of indirect negotiations have failed to produce an agreement, with Hamas rejecting the latest proposal for a six-week ceasefire in exchange for the release of 40 of the remaining 133 hostages. At least 30 hostages are presumed dead.
Israel appears to be moving ahead with plans for an offensive in Rafah, southern Gaza, despite warnings of the potentially catastrophic humanitarian consequences for the 1.5 million displaced Palestinians sheltering there.

"""
encoded_text = tokenizer.encode(new_text, return_tensors="pt")
summary_ids = model.generate(encoded_text, max_length=60)


summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(f"Generated Summary: {summary}")

Generated Summary: Hersh Goldberg-Polin has been held for almost 200 daysThe parents of an Israeli-American hostage in Gaza have urged him to "stay strong" and "survive" after Hamas released a proof-of-life video.Scroll down to watch the video.Hers


In [70]:

def predict(model, tokenizer, input_text):
    
    if isinstance(input_text, list):
        generated_summaries = []
        i = 0
        for text in input_text: 
            encoded_text = tokenizer.encode(text[:2844], return_tensors="pt")
            summary_ids = model.generate(encoded_text, max_length=60)
            summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
            generated_summaries.append(summary)
            print(i)
            i += 1
        
        return generated_summaries
        
    elif isinstance(input_text, str):
        encoded_text = tokenizer.encode(input_text, return_tensors="pt")
        summary_ids = model.generate(encoded_text, max_length=60)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary
        

data = np.load('../data/dataset_cleaned.npz', allow_pickle=True)

X_train = data['X_train']
y_train = data['y_train']
X_test = data['X_test']
y_test = data['y_test']
X_val = data['X_val']
y_val = data['y_val']

y_test = y_test.tolist()

print(len(y_test))

# Define a method to perform cross-validation manually, measuring ROUGE scores
rouge_scores = []

generated_summaries = predict(model, tokenizer, X_train.tolist())    
rouge = Rouge()    
rouge_scores = rouge.get_scores(generated_summaries, y_train.tolist())


208
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
27

AssertionError: 

In [64]:
np.savez('../data/predicted_summaires.npz', y_pred=generated_summaries, rouge_scores=rouge_scores)


In [68]:
data = np.load('../data/predicted_summaires.npz', allow_pickle=True)
y_pred = data['y_pred']
rouge_scores = data['rouge_scores']

len(y_pred)
len(rouge_scores)

208

In [73]:
r_1_f = 0
r_1_p = 0
r_1_r = 0

r_2_f = 0
r_2_p = 0
r_2_r = 0

r_l_f = 0
r_l_p = 0
r_l_r = 0

for item in rouge_scores:
    if 'rouge-1' in item.keys():
        r_1_f += item['rouge-1']['f']
        r_1_p += item['rouge-1']['p']
        r_1_r += item['rouge-1']['r']
    if 'rouge-2' in item.keys():
        r_2_f += item['rouge-2']['f']
        r_2_p += item['rouge-2']['p']
        r_2_r += item['rouge-2']['r']
    if 'rouge-l' in item.keys():
        r_l_f += item['rouge-l']['f']
        r_l_p += item['rouge-l']['p']
        r_l_r += item['rouge-l']['r']

num_items = len(X_train)

r_1_f = r_1_f / num_items
r_1_p = r_1_p / num_items
r_1_r = r_1_r / num_items

r_2_f = r_2_f / num_items
r_2_p = r_2_p / num_items
r_2_r = r_2_r / num_items

r_l_f = r_l_f / num_items
r_l_p = r_l_p / num_items
r_l_r = r_l_r / num_items


print("Rouge-1 F1_score: \t" + str(r_1_f))
print("Rouge-1 Precision: \t" + str(r_1_p))
print("Rouge-1 Recall: \t" + str(r_1_r))
print("Rouge-2 F1_score: \t" + str(r_2_f))
print("Rouge-2 Precision: \t" + str(r_2_p))
print("Rouge-2 Recall: \t" + str(r_2_r))
print("Rouge-l F1_score: \t" + str(r_l_f))
print("Rouge-l Precision: \t" + str(r_l_p))
print("Rouge-l Recall: \t" + str(r_l_r))


Rouge-1 F1_score: 	0.25952527936115555
Rouge-1 Precision: 	0.3378454350787514
Rouge-1 Recall: 	0.22676838785678186
Rouge-2 F1_score: 	0.08681028835902199
Rouge-2 Precision: 	0.11627892714562145
Rouge-2 Recall: 	0.07651952011369734
Rouge-l F1_score: 	0.1939511935768485
Rouge-l Precision: 	0.25448030861230064
Rouge-l Recall: 	0.16859916822305157
